<a href="https://colab.research.google.com/github/Gus-oss/DeepLearning/blob/main/Tarea1_Parte2_Estocastico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#Librerias importantes
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

#Coneccion con google drive
from google.colab import drive
from google.colab import files
#Importar la carpeta de drive
drive.mount('/content/drive') #coneccion con drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
df = pd.read_excel("/content/drive/MyDrive/DeepLearning/archivos/EarlyRetirementPrediction.csv.xlsx")

In [10]:
sc = StandardScaler() # Standardisación
scaled = sc.fit_transform(df)
res = pd.DataFrame(scaled, columns=df.columns) #este es el df que utilizaremos

In [11]:
# Cargar datos (asumiendo que res y df ya existen)
X_columns = ['Gender', 'Desease', 'Education Level', 'Marital Status',
             'Monthly Income', 'Employee/Employer', 'Dependants',
             'Unemployment Rate', 'Stock Market', 'Credit Score',
             'Government Bonds Return']
X_raw = res[X_columns].values
y = df['Retire Before 65 Years Old'].values.reshape(-1, 1)

In [12]:
# Funciones de activación y pérdida
def sigmoid(z):
    z = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z))

def sigmoid_prime(z):
    s = sigmoid(z)
    return s * (1 - s)

def cross_entropy_loss(y, y_hat):
    epsilon = 1e-15
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)
    return -y * np.log(y_hat)

# Inicialización de pesos
np.random.seed(42)
n_input = X_raw.shape[1]
n_hidden = 4
n_output = 1

W1 = np.random.randn(n_input, n_hidden) * 0.1
b1 = np.zeros((1, n_hidden))
W2 = np.random.randn(n_hidden, n_output) * 0.1
b2 = np.zeros((1, n_output))

# Forward propagation
def forward(X):
    z1 = X @ W1 + b1
    a1 = sigmoid(z1)
    z2 = a1 @ W2 + b2
    y_hat = sigmoid(z2)
    return z1, a1, z2, y_hat

# Backpropagation
def backward(X, y, z1, a1, z2, y_hat, eta):
    global W1, b1, W2, b2

    delta2 = (-y / y_hat) * sigmoid_prime(z2)
    dW2 = a1.T @ delta2
    db2 = delta2

    delta1 = (delta2 @ W2.T) * sigmoid_prime(z1)
    dW1 = X.T @ delta1
    db1 = delta1

    W2 -= eta * dW2
    b2 -= eta * db2
    W1 -= eta * dW1
    b1 -= eta * db1

# Desenso por gradiente estocastico
eta = 0.01
N = X_raw.shape[0]
n_epochs = 1

print("="*80)
print(f"STOCHASTIC GRADIENT DESCENT - {N} ITERACIONES")
print("="*80)

# Variables para guardar iteración 1 y 1500
iter1_info = {}
iter1500_info = {}

for epoch in range(n_epochs):

    for i in range(N):
        # Una observación a la vez
        X_single = X_raw[i:i+1]
        y_single = y[i:i+1]

        # Forward pass
        z1, a1, z2, y_hat = forward(X_single)

        # Pérdida individual
        loss = cross_entropy_loss(y_single, y_hat)[0, 0]

        # IMPRIMIR ANTES DE ACTUALIZAR
        print(f"\n{'─'*80}")
        print(f"ITERACIÓN {i+1}/{N}")
        print(f"{'─'*80}")
        print(f"Observación y = {y_single[0,0]:.0f}")
        print(f"Predicción ŷ   = {y_hat[0,0]:.6f}")
        print(f"Loss           = {loss:.6f}")

        print(f"\nPESOS CAPA OCULTA (W1) - Primeras 2 neuronas:")
        print(f"  W1[0,0] = {W1[0,0]:.8f}    W1[0,1] = {W1[0,1]:.8f}")
        print(f"  W1[1,0] = {W1[1,0]:.8f}    W1[1,1] = {W1[1,1]:.8f}")

        print(f"\nBIAS CAPA OCULTA (b1):")
        print(f"  b1 = [{b1[0,0]:.8f}, {b1[0,1]:.8f}, {b1[0,2]:.8f}, {b1[0,3]:.8f}]")

        print(f"\nPESOS CAPA SALIDA (W2):")
        print(f"  W2 = [{W2[0,0]:.8f}, {W2[1,0]:.8f}, {W2[2,0]:.8f}, {W2[3,0]:.8f}]ᵀ")

        print(f"\nBIAS CAPA SALIDA (b2):")
        print(f"  b2 = {b2[0,0]:.8f}")

        # Guardar información de iteración 1
        if i == 0:
            iter1_info = {
                'i': i+1,
                'y': y_single[0,0],
                'y_hat': y_hat[0,0],
                'loss': loss,
                'W1_00': W1[0,0],
                'W1_01': W1[0,1],
                'b1_0': b1[0,0],
                'W2_0': W2[0,0],
                'b2': b2[0,0]
            }

        # Backward pass y actualización
        backward(X_single, y_single, z1, a1, z2, y_hat, eta)

        # Guardar información de iteración 1500 (después de actualizar)
        if i == N - 1:
            # Recalcular forward con pesos actualizados
            z1_new, a1_new, z2_new, y_hat_new = forward(X_single)
            loss_new = cross_entropy_loss(y_single, y_hat_new)[0, 0]

            iter1500_info = {
                'i': i+1,
                'y': y_single[0,0],
                'y_hat': y_hat_new[0,0],
                'loss': loss_new,
                'W1_00': W1[0,0],
                'W1_01': W1[0,1],
                'b1_0': b1[0,0],
                'W2_0': W2[0,0],
                'b2': b2[0,0]
            }

# Comparación final
print("\n" + "="*80)
print("COMPARACIÓN: ITERACIÓN 1 vs ITERACIÓN 1500")
print("="*80)

print(f"\n{'MÉTRICA':<25} {'ITERACIÓN 1':>20} {'ITERACIÓN 1500':>20} {'CAMBIO':>15}")
print("─"*80)

print(f"{'y (real)':<25} {iter1_info['y']:>20.0f} {iter1500_info['y']:>20.0f} {'-':>15}")
print(f"{'ŷ (predicción)':<25} {iter1_info['y_hat']:>20.6f} {iter1500_info['y_hat']:>20.6f} {iter1500_info['y_hat']-iter1_info['y_hat']:>15.6f}")
print(f"{'Loss':<25} {iter1_info['loss']:>20.6f} {iter1500_info['loss']:>20.6f} {iter1500_info['loss']-iter1_info['loss']:>15.6f}")

print(f"\n{'PESOS CAPA OCULTA':<25}")
print("─"*80)
print(f"{'W1[0,0]':<25} {iter1_info['W1_00']:>20.8f} {iter1500_info['W1_00']:>20.8f} {iter1500_info['W1_00']-iter1_info['W1_00']:>15.8f}")
print(f"{'W1[0,1]':<25} {iter1_info['W1_01']:>20.8f} {iter1500_info['W1_01']:>20.8f} {iter1500_info['W1_01']-iter1_info['W1_01']:>15.8f}")
print(f"{'b1[0]':<25} {iter1_info['b1_0']:>20.8f} {iter1500_info['b1_0']:>20.8f} {iter1500_info['b1_0']-iter1_info['b1_0']:>15.8f}")

print(f"\n{'PESOS CAPA SALIDA':<25}")
print("─"*80)
print(f"{'W2[0]':<25} {iter1_info['W2_0']:>20.8f} {iter1500_info['W2_0']:>20.8f} {iter1500_info['W2_0']-iter1_info['W2_0']:>15.8f}")
print(f"{'b2':<25} {iter1_info['b2']:>20.8f} {iter1500_info['b2']:>20.8f} {iter1500_info['b2']-iter1_info['b2']:>15.8f}")

print("\n" + "="*80)
print("ANÁLISIS")
print("="*80)
print(f"Reducción de Loss: {((iter1_info['loss']-iter1500_info['loss'])/iter1_info['loss']*100):.2f}%")
print(f"Total de actualizaciones de pesos: {N}")
print("="*80)

STOCHASTIC GRADIENT DESCENT - 1500 ITERACIONES

────────────────────────────────────────────────────────────────────────────────
ITERACIÓN 1/1500
────────────────────────────────────────────────────────────────────────────────
Observación y = 1
Predicción ŷ   = 0.479982
Loss           = 0.734006

PESOS CAPA OCULTA (W1) - Primeras 2 neuronas:
  W1[0,0] = 0.04967142    W1[0,1] = -0.01382643
  W1[1,0] = -0.02341534    W1[1,1] = -0.02341370

BIAS CAPA OCULTA (b1):
  b1 = [0.00000000, 0.00000000, 0.00000000, 0.00000000]

PESOS CAPA SALIDA (W2):
  W2 = [-0.14785220, -0.07198442, -0.04606388, 0.10571222]ᵀ

BIAS CAPA SALIDA (b2):
  b2 = 0.00000000

────────────────────────────────────────────────────────────────────────────────
ITERACIÓN 2/1500
────────────────────────────────────────────────────────────────────────────────
Observación y = 0
Predicción ŷ   = 0.483397
Loss           = -0.000000

PESOS CAPA OCULTA (W1) - Primeras 2 neuronas:
  W1[0,0] = 0.04951655    W1[0,1] = -0.01390050
  W1[1